<a href="https://colab.research.google.com/github/lqh52/Early-prediction-of-sepsis/blob/main/Feature_engineering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#library
import pandas as pd
import numpy as np
import copy
from sklearn.impute import KNNImputer
from sklearn import preprocessing
#load data
df = pd.read_pickle('/content/drive/MyDrive/Sepsis_predict/data.pkl')

In [ ]:
#drop Unit2 because Unit1 and Unit2 are mutually exclusive
#drop ICULOS as it's basically just an index
cols_to_drop = ['Unit2', 'ICULOS']
df = df.drop(cols_to_drop, axis=1)

In [ ]:
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,HospAdmTime,SepsisLabel,patient,qsofa,sofa,SIRS,ShockIndex,BUN/CR,mews,sofa_det
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,-0.03,0,000001,0,0,0,NaN,NaN,0,0
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,-0.03,0,000001,1,0,0,0.989796,NaN,2,0
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,-0.03,0,000001,1,0,0,0.729508,NaN,2,0
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,NaN,7.36,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,-0.03,0,000001,1,0,0,NaN,NaN,3,0
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,-0.03,0,000001,1,0,1,0.844262,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,76.0,99.0,37.3,137.0,98.00,79.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.00,0,0.0,-13.25,0,109334,1,0,0,0.554745,NaN,2,0
1552206,90.0,100.0,NaN,116.0,88.00,70.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.00,0,0.0,-13.25,0,109334,1,0,0,0.775862,NaN,2,0
1552207,90.0,100.0,NaN,118.0,84.00,64.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.00,0,0.0,-13.25,0,109334,0,0,0,0.762712,NaN,1,0
1552208,82.0,100.0,NaN,114.0,83.00,66.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.00,0,0.0,-13.25,0,109334,0,0,0,0.719298,NaN,1,0


In [ ]:
non_continous_data = ['Age', 'HospAdmTime', 'SepsisLabel', 'sofa_det', 'Gender', 'SIRS', 'Unit1']
continous_data = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 
                  'pH', 'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 
                  'Bilirubin_direct','Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 
                  'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'qsofa', 'sofa', 'ShockIndex', 'BUN/CR', 'mews']
len(non_continous_data)

7

In [ ]:
#Random pick 8000 patient to test set
patients_training_data = df['patient'].unique()
np.random.shuffle(patients_training_data)
patients_training_data = patients_training_data[0:-8000]

In [ ]:
#calculate mean of each feature
df_mean = df[df['patient'].isin(patients_training_data)].describe().loc[['mean']]
df_mean.to_pickle('df_mean.pkl')

In [ ]:
#loop through each patient at a time
imputer = KNNImputer()
save_count = 0
df_list = []
grouped_by_patient = df.groupby('patient')
columns = df.columns
for patient, group in grouped_by_patient:
    #print(patient)
    group = group.reset_index(drop=True)

    #fill completely missing data with mean of dataset
    missing_rate = group.isna().sum()/len(group)
    missing_rate = missing_rate.to_dict()
    for stat in columns:
      if missing_rate[stat] == 1:
        group.loc[:,stat] = df_mean[stat][0]
    
    #fill other missing data with KNN
    group = pd.DataFrame(imputer.fit_transform(group.values), columns=columns)

    #sepsis occurence and time
    group['sepsis_occur'] = -1
    group['time_to_sepsis'] = -1
    
    #generate windows of 6 hours
    N = len(group)
    win_len = 6
    i = 0
    while(i+win_len <= N):
      tmp_data = group.iloc[i:i+win_len]
      if tmp_data[tmp_data['SepsisLabel']==1].empty:
        group.loc[i,'sepsis_occur'] = 0
        group.loc[i,'time_to_sepsis'] = 7
      else:
        group.loc[i,'sepsis_occur'] = 1
        group.loc[i,'time_to_sepsis'] = tmp_data[tmp_data['SepsisLabel']==1].index.values[0] - i
      
      #slide window 
      i += 1
    
    while(i<N):
      tmp_data = group.iloc[i:N]
      if tmp_data[tmp_data['SepsisLabel']==1].empty:
        group.loc[i,'sepsis_occur'] = 0
        group.loc[i,'time_to_sepsis'] = 7
      else:
        group.loc[i,'sepsis_occur'] = 1
        group.loc[i,'time_to_sepsis'] = tmp_data[tmp_data['SepsisLabel']==1].index.values[0] - i
      i += 1

    
    df_list.append(group)
    save_count += 1
    if (save_count%5000==0):
      print('Saved: ',save_count)
      
new_df = pd.concat(df_list).reset_index(drop=True)

Saved:  5000
Saved:  10000
Saved:  15000
Saved:  20000
Saved:  25000
Saved:  30000
Saved:  35000
Saved:  40000


In [ ]:
train = new_df[new_df['patient'].isin(patients_training_data)].drop('patient', axis=1)
test = new_df[~new_df['patient'].isin(patients_training_data)].drop('patient', axis=1)

In [ ]:
#Scale continuos data
train[continous_data] = (train[continous_data] - train[continous_data].min(axis=0))/(train[continous_data].max(axis=0)-train[continous_data].min(axis=0))
test[continous_data] = (test[continous_data] - test[continous_data].min(axis=0))/(test[continous_data].max(axis=0)-test[continous_data].min(axis=0))

In [ ]:
#Scale discrete data
train[['Age', 'HospAdmTime']] = (train[['Age', 'HospAdmTime']] - train[['Age', 'HospAdmTime']].min(axis=0))/(train[['Age', 'HospAdmTime']].max(axis=0)-train[['Age', 'HospAdmTime']].min(axis=0))
test[['Age', 'HospAdmTime']] = (test[['Age', 'HospAdmTime']] - test[['Age', 'HospAdmTime']].min(axis=0))/(test[['Age', 'HospAdmTime']].max(axis=0)-test[['Age', 'HospAdmTime']].min(axis=0))

In [ ]:
#save to pickle file
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')